In [ ]:
!pip install lat-lon-parser

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from lat_lon_parser import parse

In [ ]:
## Webscrabe from Wiki information for three cities
def cities_dataframe(cities):
  cities_data = []

  for city in cities:
    city_data = {}

    # city
    city_data["City"] = city

    # create the soup
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # country
    city_data["Country"] = city_soup.find(class_="infobox-data").get_text()

    # population
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    city_data["Population"] = city_population_clean

    # data retrieved
    city_data["Year_Data_Retrieved"] = city_soup.find(string="Population").find_next().get_text()[2:6]

    # latitude and longitude
    city_data["Latitude"] = city_soup.find(class_="latitude").get_text()
    city_data["Longitude"] = city_soup.find(class_="longitude").get_text()
    city_data["Latitude"] = parse(city_data["Latitude"])
    city_data["Longitude"] = parse(city_data["Longitude"])

    # append this city's data to the cities list
    cities_data.append(city_data)

  return pd.DataFrame(cities_data)

In [ ]:
cities_df = cities_dataframe(["Berlin", "Hamburg", "Munich"])
cities_df

In [ ]:
# Create the "cities df" for the relational data base
cities_to_db = cities_df[["City", "Country", "Latitude", "Longitude"]].copy()

In [ ]:
cities_to_db.rename(columns={"City": "City_name"}, inplace=True)
cities_to_db

In [ ]:
cities_to_db.to_csv('cities_to_db.csv', index=False)

In [ ]:
# Create the "population df" for the relational data base
population_to_db = cities_df[["Population", "Year_Data_Retrieved"]]
population_to_db

In [ ]:
# Establishment of connection with the SQL database
schema = "gans_local" # The name of your database
host = "localhost"
user = "root"
password = MySQL_pass
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
# Push the "cities_to_db" to the empty "cities" table in the MySQL data base
cities_to_db.to_sql('cities', if_exists='append', con=connection_string, index=False)

In [ ]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql 

In [ ]:
# Getting the "City_id" to the population df
population_to_db = population_to_db.copy()
population_to_db["City_id"] = cities_from_sql["City_id"]
population_to_db

In [ ]:
population_to_db.to_csv('population_to_db.csv', index=False)

In [ ]:
# Push the "population_to_db" to the empty "population" table in the MySQL data base
population_to_db.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)